Recursion is a technique by which a function makes one or more calls to itself during execution.

## 1. Illustrative Examples

### 1.1 Factorial Function

$$n!=
\begin{cases}
1& \text{if n = 0}\\
n*(n-1)*(n-2)...*3*2*1& \text{if n >= 1}
\end{cases}$$

<br>
Recursive definition:
<br><br>
$$n!=
\begin{cases}
1& \text{if n = 0}\\
n*(n-1)!& \text{if n >= 1}
\end{cases}$$

<br>
Typical recursive definition:

    - contains one or more base cases, defined nonrecursively
    - contains one ore more recursive cases, defined by appealing to the definition of the function being defined.

In [1]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n*factorial(n-1)

In Python, each time a function is called, a structure known as an **activation record** or **frame** is created to store information about the progress of that invocation of the function.
<br><br>
This activation record includes a namespace for stroing the function call's parameters and local variables, information about which command in the body of the function is currently executing.
<br><br>
When the execution of a function leads to a nested function call, the execution of the former call is suspended, and its activation record stores the place in the source code at which the flow of control should continue upon return of the nested call.
<br><br>
There is a different activation record for each active call.

### 1.2 English Ruler

In general, an interval with a central tick length L >= 1 is composed of:
    - An interval with a central tick length L-1
    - An single tick of length L
    - An interval with a central tick lenth L-1

In [5]:
def draw_line(tick_length, tick_label=''):
    '''
    Draw one line with given tick length
    '''
    line = '-'*tick_length
    if tick_label:
        line += ' ' + tick_label
    print(line)
    
def draw_interval(center_length):
    '''
    Draw tick interval based upon a central tick length
    '''
    if center_length > 0:
        draw_interval(center_length - 1)
        draw_line(center_length)
        draw_interval(center_length - 1)

def draw_ruler(num_inches, major_length):
    '''
    Draw English ruler with given number of inches, major tick length
    '''
    draw_line(major_length, '0')
    for j in range(1, 1 + num_inches):
        draw_interval(major_length - 1)
        draw_line(major_length, str(j))

In [7]:
draw_ruler(2,4)

---- 0
-
--
-
---
-
--
-
---- 1
-
--
-
---
-
--
-
---- 2


### 1.3 Binary Search

O(logn)

In [8]:
def binary_search(data, target, low, high):
    '''
    The search only considers the portion from data[low] to data[high] inclusive
    '''
    if low > high:
        return False
    mid = (low + high) // 2
    if target == data[mid]:
        return True
    elif target < data[mid]:
        return binary_search(data, target, low, mid - 1)
    else:
        return binary_search(data, target, mid + 1, high)

In [9]:
data = [2,4,5,7,8,9,12,13,17,19,22,25,27,8,33,37]
binary_search(data, 22, 0, len(data)-1)

True

### 1.4 File Systems

Pseudo-code:
<br>
Algorithm DiskUsage(path):
    - Input: A string designating a path to a file-system entry
    - Output: The cumulative disk space used by that entry and any nested entries
    - total = size(path) {immediate disk space used by the entry}
    - if path represents a directory then
        for each child entry stored within directory path do
            total = total + DiskUsage(child)
    - return total

In [10]:
import os

def disk_usage(path):
    '''
    Return the number of bytes used by a file/folder and any descendents
    '''
    total = os.path.getsize(path) # account for direct usage
    if os.path.isdir(path):
        for filename in os.listdir(path):
            childpath = os.path.join(path, filename) # compose full path to child
            total += disk_usage(childpath)
    print('{0:<7}'.format(total), path)
    return total

## 2. Analyzing Recursive Algorithms

    For each invocation of the function, only account for the number of operations that are performed within the body of that activation.
    Then account for the overall number of operations by taking the sum

### 2.1 Factorials

There are total n+1 activations, and each individual activation executes a constant number of operations.
<br>
O(n)

### 2.2 English Ruler

How many total lines of output are generated by an initial call to draw_interval(c):
<br><br>
$$2^c - 1$$

### 2.3 Binary Search

With each recursive call the number of candidate entries still to be searched is given by the value:
    
    high - low + 1

The number of remaining candidates is reduced by at least on half with each recursive call:

    (mid-1) - low + 1 = (high + low)/2 - low <= (high - low + 1)/2
    high - (mid+1) + 1 = high - high + low)/2 <= (high - low + 1)/2

In the worst case, the number of recursive calls performed is the smallest integer r such that:
<br><br>
$$\frac{n}{2^r} < 1$$
<br>
$$r>logn \;\;-> r = logn + 1$$
<br>
The binary search runs in O(logn) time

## 3. Recursion Run Amok

### 3.1 element uniqueness problem

When n = 1, the elements are unique.
<br>
When n >= 2, the elements are unique if and only if:
    - the first n-1 elements are unique
    - the last n-1 eleements are unique
    - the first and last elements are different

In [11]:
def unique3(S, start, stop):
    '''
    Return True if there are no duplicate elements in slice S[start:stop]
    '''
    if stop - start <= 1: # at most one item
        return True
    if not unique(S, start, stop-1): 
        return False
    if not unique(S, start+1, stop):
        return False
    return S[start] != S[stop-1]

The nonrecursive part of each call uses O(1).<br>
A single call to unique3 of size n will result in two recursive call of size n-1, in the worst case, the total number of function calls:
<br>
$$1 + 2 + 4 + 8 + ...+2^{n-1}$$
<br>
So the running time is O($2^n$)

### 3.2 Fibonacci problem

In [16]:
def bad_fibonacci(n):
    if n <= 1:
        return n
    return bad_fibonacci(n-2) + bad_fibonacci(n-1)

In this way, the number of calls is exponential in n.<br>
Because after computing $F_{n-2}$, the call to compute $F_{n-1}$ requires its own recursive call to compute $F_{n-2}$, which is duplicative work.

In [17]:
def original_fibonacci(n):
    if n <= 1:
        return n
    a = 0
    b = 1
    for i in range(n-1):
        a,b = b,a+b
    return b

Passing the F(n-1) from one level of the recursion to the next makes it much easier to continue the process.

In [18]:
# O(n)
def good_fibonacci(n):
    '''
    return pair of Fibonacci numbers, F(n) and F(n-1)
    '''
    if n <= 1:
        return (n, 0)
    (a,b) = good_fibonacci(n-1)
    return (a+b, a)

### 3.3 Maximum Recursive Depth in Python

Inifinite recursion: 
    - recursive call never reaching a base case.
    
Typically, if the overall number of simultaneously function activations is over 1000, Python will raise a RuntimeError with a message:
    - maximum recursion depth exceeded
    
This can be changed by:

In [19]:
#import sys
#old = sys.getrecursionlimit()  # 1000 is typical
#sys.setrecursionlimit(1000000)

## 4. Further Examples of Recursion

### 4.1 Linear Recursion

**Linear Recursion:**
    If a recursive call starts at most one other
    
#### Summing the Elements of a Sequence Recursively

The sum of all n integers in S is 0, if n = 0
<br>
Otherwise it is the sum of the first n-1 element plus the last element

In [20]:
# O(n), n+1 function calls and constant operation in each call
def linear_sum(S, n):
    if n == 0:
        return 0
    return linear_sum(S, n-1) + S[n-1]

#### Reverse a Sequence with Recursion

In [21]:
# O(n), 1+n/2 recursive calls
# When calling the function stop = len(S)
def reverse(S, start, stop):
    '''
    Reverse elements in slice S[start:stop]
    '''
    if start < stop - 1:
        S[start],S[stop-1] = S[stop-1],S[start]
        reverse(S,start+1,stop-1)

#### Recursive Algorithms for Computing Powers

$$power(x,n)=
\begin{cases}
1& \text{if n = 0}\\
x*power(x,n-1)& \text{otherwise}
\end{cases}$$


In [22]:
# O(n)
def power(x,n):
    if n == 0:
        return 1
    return x * power(x, n-1)

$$power(x,n)=
\begin{cases}
1& \text{if n = 0}\\
x * (power(x,n//2))^2& \text{if n > 0 and n is odd}\\
(power(x,n//2))^2& \text{if n > 0 and n is even}
\end{cases}$$

In [25]:
# O(logn)
def power(x,n):
    if n == 0:
        return 1
    p = power(x, n//2)**2
    if n % 2 == 1:
        p *= x
    return p

### 4.2 Binary Recursion

**Binary Recursion:**
    - if each invocation of a recursive function makes two new recursive call.

In [26]:
# O(n), 2n-1 function calls
def binary_sum(S, start, stop):
    '''
    Return the sum of the numbers in implicit slice S[start:stop]
    '''
    if start >= stop:
        return 0
    if start == stop - 1:
        return S[start]
    mid = (start + stop) // 2
    return binary_sum(S, start, mid) + binary_sum(S, mid, stop)

The depth of the recursion is 1 + $log_2n$, 
<br>so the amount of additional space is O(logn), 
<br>which is a big improvement over the O(n) by the linear_sum function.

### 4.3 Multiple Recursion

**Multiple Recursion: **
    - A function make more than two recursive calls

## 5. Designing Recursive Algorithm

An algorithm that uses recursion typically has the form:
    - Test for base cases. At least one
    - Recur. Perform when not a base case
    
### Parameterizing a Recursion

Reparameterizing the signature of the function.
    - Redefine the original problem to facilitate similar-looking subproblems.

binary_search(data, target) -> binary_search(data, target, low, high)

## 6. Eliminating Tail Recursion

Recursive approach:
    - take advantage of a repetitive structure present in many problems
    - Must maintain activation records that keep track of the state of each nested call


**Tail Recursion**:
    - Eliminated without any use of axillary memory
    - Any recursive call that is made from one context is the very last operation in that context, with the return value of the recursive call immediately returned by the enclosing recursion.
    - Must be a linear recursion
    
Factorial is not a tail recursion: 
    - n * factorial(n-1)
    - an additional multiplication is performed after the recursive call is completed.
    
Any tail recursion can be reimplemented nonrecursively by enclosing the body in a loop for repetition.

In [27]:
def binary_search_iterative(data, target):
    low = 0
    high = len(data)-1
    while low <= high:
        mid = (low + high) // 2
        if target == data[mid]:
            return True
        elif target < data[mid]:
            high = mid - 1
        else:
            low = mid + 1
    return False

In [28]:
def reverse(S, start, stop):
    '''
    Reverse elements in slice S[start:stop]
    '''
    if start < stop - 1:
        S[start],S[stop-1] = S[stop-1],S[start]
        reverse(S,start+1,stop-1)
def reverse_iterative(S):
    start = 0
    stop = len(S) - 1
    while start < stop - 1:
        S[start],S[stop-1] = S[stop-1],S[start]
        start, stop = start+1, stop-1